In [ ]:
from addons.stock_manage.models.stock_manage import StocksBase
from addons.stock_manage.models.stock_manage import StockDaily

res = (StockDaily.select(StockDaily.ts_code, 
                         StockDaily.trade_date, 
                         StockDaily.open,
                        StockDaily.high, 
                        StockDaily.low, 
                        StockDaily.close, 
                        StockDaily.pre_close, 
                        StockDaily.pct_chg,
                        StockDaily.change,StockDaily.vol, StockDaily.amount).
        where((StockDaily.ts_code == "300059.SZ") & 
        (StockDaily.trade_date.between('2020-01-01', '2023-03-23'))).order_by(StockDaily.trade_date))
print(res)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

stock_df = pd.DataFrame(list(res.dicts()))


In [ ]:
stock_df.head()


In [ ]:
stock_df = stock_df.set_index('trade_date',drop=False, append=False, inplace=False, verify_integrity=False)

In [ ]:
# stock_df_signal = pd.DataFrame(index = stock_df.index)
# stock_df_signal["price"] = stock_df["close"]
# stock_df_signal["diff"] = stock_df_signal["price"].diff()
# stock_df_signal = stock_df_signal.fillna(0)
# stock_df_signal["signal"] = np.where(stock_df_signal['diff'] >= 0, 0, 1)
# stock_df_signal["order"] = stock_df_signal['signal'].diff() * 100
# stock_df_signal.head()
stock_df.head()

In [ ]:
#首先导入鸢尾花数据载入工具
from sklearn.model_selection import train_test_split
#导入KNN分类模型
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
#为了方便可视化，我们再导入matplotlib和seaborn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
max_min_scaler = lambda x: (x-np.min(x)) / (np.max(x) - np.min(x))

In [ ]:
#由于我们通过股票价格变化的情况对数据进行了分类，即1代表价格上涨，–1代表价格下跌或不变，这个交易条件可以用来训练分类模型
def classification_tc(df: pd.DataFrame):
    df["Open-Close"] = df["open"] - df["close"]
    df["High-Low"] = df["high"] - df["low"]
    df["Val_Norm"] = df[["vol"]].apply(max_min_scaler)
    df["Amount_Norm"] = df[["amount"]].apply(max_min_scaler)
    # 次日收盘价高于当日收盘价
    df["target"] = np.where(df["close"].shift(-1) > df["close"], 1, -1)
    df.fillna(0, inplace=True)
    X = df[['Open-Close', 'High-Low', 'Val_Norm', 'Amount_Norm']]
    y = df["target"]
    return (X, y)

#使用classification_tc函数生成数据集的特征与目标
X, y = classification_tc(stock_df)
#将数据集拆分为训练集与验证集
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
#创建一个KNN实例，n_neighbors取95
knn_clf = KNeighborsClassifier(n_neighbors=95)
#使用KNN拟合训练集
knn_clf.fit(X_train, y_train)
#输出模型在训练集中的准确率
print(knn_clf.score(X_train, y_train))
#输出模型在验证集中的准确率
print(knn_clf.score(X_test, y_test))

stock_df['Predict_Signal'] = knn_clf.predict(X)
#在数据集中添加一个字段，用当日收盘价除以前一日收盘价，并取其自然对数
stock_df['Return'] = np.log(stock_df['close']/stock_df['pre_close'])
#查看一下
print(stock_df.tail())


In [ ]:
# # 如果要创建用于回归模型的交易条件，则可以对代码稍做调整，将次日收盘价减去当日收盘价的差作为预测的目标
# # 这样就可以训练回归模型，使其预测次日股价上涨（或下跌）的幅度。
# def regression_tc(df: pd.DataFrame):
#     df["Open-Close"] = df["open"] - df["close"]
#     df["High-Low"] = df["high"] - df["low"]
#     df["target"] = df["close"].shift(-1) - df["close"]
#     df["Val_Norm"] = df[["vol"]].apply(max_min_scaler)
#     df["Amount_Norm"] = df[["amount"]].apply(max_min_scaler)
#     df.fillna(0, inplace=True)
#     X = df[['Open-Close', 'High-Low', 'Val_Norm', 'Amount_Norm']]
#     y = df["target"]
#     return (X, y)
# #创建一个KNN实例，n_neighbors取95
# X, y = regression_tc(stock_df)
# #将数据集拆分为训练集与验证集
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
# knn_reg = KNeighborsRegressor(n_neighbors=95)
# #使用KNN拟合训练集
# knn_reg.fit(X_train, y_train)
# #输出模型在训练集中的准确率
# print(knn_reg.score(X_train, y_train))
# #输出模型在验证集中的准确率
# print(knn_reg.score(X_test, y_test))

# # 使用模型预测作为交易信号（trading signal）来进行交易，并且与基准收益进行对比。
# # 首先我们要计算出基准收益和基于模型预测的策略所带来的收益。
# #使用KNN模型预测每日股票的涨跌，保存为Predict_Signal
# stock_df['Predict_Signal'] = knn_clf.predict(X)
# #在数据集中添加一个字段，用当日收盘价除以前一日收盘价，并取其自然对数
# stock_df['Return'] = np.log(stock_df['close']/stock_df['pre_close'])
# #查看一下
# print(stock_df.tail())

In [ ]:
# 计算一下累计的基准收益。
def cum_return(df, split_value):
    cum_return = df[split_value:]['Return'].cumsum() * 100
    return cum_return


#计算基于KNN模型预测的交易信号所进行的策略交易带来的收益
def stragegy_return(df, split_value):
    df["Strategy_Return"] = df["Return"] * df["Predict_Signal"].shift(1)
    # 将每日策略交易的收益加并乘100
    cum_strategy_return = df[split_value:]["Strategy_Return"].cumsum() * 100
    return cum_strategy_return

In [ ]:
#定义一个进行可视化的函数
def plot_chart(cum_return, cum_strategy_return, symbol):
    plt.figure(figsize=(9, 6))
    plt.plot(cum_return, '--', label='%s Return'%symbol)
    plt.plot(cum_strategy_return, label='Strategy Return')
    plt.legend()
    plt.show()

In [ ]:
cum_return = cum_return(stock_df, len(X_train))
cum_strategy_return = stragegy_return(stock_df, len(X_train))
plot_chart(cum_return, cum_strategy_return, "zgpa")